In [2]:
# import liblary
import torch
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
from src.data import get_datasetCML, data_semi_learning
from src.logging_config import logger
from src.model.model import (
    LSTMModel,
    LeNet,
    LeNet_5,
    resnet20,
    resnet32,
    resnet44,
    resnet56,
    resnet110,
    resnet1202,
)
from src.utils import *

MODEL_DICT = {
    "LSTMModel": LSTMModel,
    "LeNet": LeNet,
    "LeNet_5": LeNet_5,
    "resnet20": resnet20,
    "resnet32": resnet32,
    "resnet44": resnet44,
    "resnet56": resnet56,
    "resnet110": resnet110,
    "resnet1202": resnet1202,
}

## Visualization Data
1. First train model with 5 classes of Data (Cifar10)
epoch [1, 5, 10, 20, 40, 50] --> acc increase
--> Save model parameter 

2. Visualized
Data with class (5 + n) collect outputs and visulized 

In [8]:
# trainning model with 5 classes [0,1,2,3,4]
data_use = "cifar10"
num_classes = 10
batch_size = 64
labeled_classed = [0,1,2,3,4]

trainset_5class, trainset_unlabeled = data_semi_learning(data_use=data_use, num_class=num_classes, batch_size=64, labeled_classes=labeled_classed)

for batch in trainset_5class:
    print(f"check check the batch: {batch[0]}:\n")
    # print(f"check check the idx: {idx}")
    # print(f"check check the idx: {data}")
    break

# for batch in trainset_unlabeled:
#     print(f"check check the batch: {batch[0]}:\n")
#     # print(f"check check the idx: {idx}")
#     # print(f"check check the idx: {data}")
#     break


Files already downloaded and verified
Files already downloaded and verified
check check the batch: tensor([[[-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
         [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
         [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
         ...,
         [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
         [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
         [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291]],

        [[-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
         [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
         [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
         ...,
         [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
         [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
         [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183]],

        [[-2.2214

In [ ]:
trainloader5classes = DataLoader(trainset_5class, batch_size=64, shuffle=True)

## trainning
def train_one_epoch(model, trainloader, optimizer, criterion, device, epoch):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    progress_bar = tqdm(trainloader, desc=f"Epoch {epoch+1}", leave=True)

    for idx, (inputs, labels) in enumerate(progress_bar):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs, _ = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    if epoch in [1, 5, 10, 20, 40, 50]:
        torch.save(model.state_dict(), f"Lenet5_cifar10_final_{epoch}.pth")

    epoch_loss = running_loss / len(trainloader)
    accuracy = 100 * correct / total

    logger.info(f"Epoch [{epoch+1}] | Loss: {epoch_loss:.4f} | Accuracy: {accuracy:.2f}%")

def trainning_main():
    device = torch.device("cuda" if torch.cuda.is_available else "cpu")
    model_run = "LeNet"
    model= MODEL_DICT.get(model_run, None)
    


## Create Pseudo Label
1. Check with cluster algorithsm
- DBSCAN
- Affinity Propagation
-